In [2]:
import numpy as np
import pandas as pd
import pandas as pd
import re
from konlpy.tag import Okt
import pandas as pd
import re

beauty_df = pd.read_csv('Beauty_kurly.txt', encoding = 'utf-8')
beauty_df.head()

,title,nickname,datetime,content
0,뷰티컬리페스타 추천템 세일 달려요!,에일리,2023. 2. 11. 18:35,뷰티\n뷰티컬리페스타 추천템 세일 달려요!\n에일리 ・ 2023. 2. 11. 18...
1,마켓컬리 화장품 뷰티컬리 러쉬 입욕제,진,2022. 11. 1. 23:52,리뷰\n마켓컬리 화장품 뷰티컬리 러쉬 입욕제\n진 ・ 2022. 11. 1. 23:...
2,뷰티컬리 오픈이벤트 챙겨 헤라 블랙쿠션 샛별배송 받았다!,라하이맘,2022. 11. 13. 23:37,나를위한 뷰티\n뷰티컬리 오픈이벤트 챙겨 헤라 블랙쿠션 샛별배송 받았다!\n라하이맘...
3,뷰티컬리 입점템 라로제 토닉로션으로 5초 수분 생기 채워요,감성 쿠굥이,2023. 8. 8. 20:10,뷰티\n뷰티컬리 입점템 라로제 토닉로션으로 5초 수분 생기 채워요\n감성 쿠굥이 ・...
4,뷰티컬리 페스타 역대급 세일 내일까지 모두 달려욧,미셸,2023. 2. 15. 21:24,etc.\n뷰티컬리 페스타 역대급 세일 내일까지 모두 달려욧\n미셸 ・ 2023. ...


In [3]:
def preprocess_text(text):
    # 1. 불필요한 메타데이터 제거
    text = text.split('인쇄')[0]

    # 2. 줄바꿈 및 들여쓰기 제거
    text = text.replace('\n', ' ').strip()

    # 3. 텍스트 정규화
    text = text.lower() # 모두 소문자로 변환
    text = re.sub(r'[^\w\s]', ' ', text) # 문장 부호 및 특수 문자 제거

    # 4. Tokenization and stemming
    okt = Okt()
    tokens = okt.morphs(text)

    # 5. 불용어 제거
    stopwords = ['이', '그', '것', '의', '을', '를', '와', '과']
    filtered_tokens = [word for word in tokens if word not in stopwords]

    return ' '.join(filtered_tokens)

# 해시태그 추출 함수
def extract_hashtags(text):
    hashtags = re.findall(r'\n#\w+', text)
    return hashtags

# 공감 수 추출 함수
def extract_likes(text):
    match = re.search(r'\n공감 (\d+)', text)
    if match:
        return int(match.group(1))
    return None

# 댓글 수 추출 함수
def extract_comments(text):
    match = re.search(r'\n댓글 (\d+)', text)
    if match:
        return int(match.group(1))
    return None

# 해시태그 칼럼 전처리
def hashtag_split(text):
    text = text.replace('\n#','')
    return text

def del_oddsments(list_):
    new_list = list(map(hashtag_split, list_))
    return new_list

# 새로운 칼럼 추가
beauty_df['hashtags'] = beauty_df['content'].apply(extract_hashtags)
beauty_df['likes'] = beauty_df['content'].apply(extract_likes)
beauty_df['comments'] = beauty_df['content'].apply(extract_comments)
# 결측치 처리 및 해시태그 '\n#' 삭제
beauty_df['hashtags'] = beauty_df['hashtags'].apply(del_oddsments)
beauty_df['likes'] = beauty_df['likes'].fillna(0)
beauty_df['comments'] = beauty_df['comments'].fillna(0)

beauty_df['preprocessed_content'] = beauty_df['content'].apply(preprocess_text)

In [4]:
beauty_df.head()

,title,nickname,datetime,content,hashtags,likes,comments,preprocessed_content
0,뷰티컬리페스타 추천템 세일 달려요!,에일리,2023. 2. 11. 18:35,뷰티\n뷰티컬리페스타 추천템 세일 달려요!\n에일리 ・ 2023. 2. 11. 18...,"[뷰티컬리페스타, 뷰티컬리, 뷰티컬리세일, 뷰티컬리추천, 뷰티컬리추천템, 뷰티]",24.0,1.0,뷰티 뷰티 컬 리페 스타 추천 템 세 일 달려요 에일리 2023 2 11 18 35...
1,마켓컬리 화장품 뷰티컬리 러쉬 입욕제,진,2022. 11. 1. 23:52,리뷰\n마켓컬리 화장품 뷰티컬리 러쉬 입욕제\n진 ・ 2022. 11. 1. 23:...,"[마켓컬리, 마켓컬리추천, 마켓컬리화장품, 뷰티컬리, 뷰티컬리러쉬, 러쉬배쓰밤, 러...",0.0,0.0,리뷰 마켓 컬리 화장품 뷰티 컬리 러쉬 입욕 제 진 2022 11 1 23 52 u...
2,뷰티컬리 오픈이벤트 챙겨 헤라 블랙쿠션 샛별배송 받았다!,라하이맘,2022. 11. 13. 23:37,나를위한 뷰티\n뷰티컬리 오픈이벤트 챙겨 헤라 블랙쿠션 샛별배송 받았다!\n라하이맘...,"[헤라블랙쿠션, 뷰티컬리, 마켓컬리, 샛별배송, 헤라쿠션]",23.0,3.0,나를 위 한 뷰티 뷰티 컬리 오픈 이벤트 챙겨 헤라 블랙 쿠션 샛별 배송 받았다 라...
3,뷰티컬리 입점템 라로제 토닉로션으로 5초 수분 생기 채워요,감성 쿠굥이,2023. 8. 8. 20:10,뷰티\n뷰티컬리 입점템 라로제 토닉로션으로 5초 수분 생기 채워요\n감성 쿠굥이 ・...,"[뷰티컬리입점템, 마켓컬리추천상품, 라로제, 라로제토닉로션, 라로제토너, 뷰티컬리]",10.0,1.0,뷰티 뷰티 컬리 입점템 라 로제 토닉 로션 으로 5초 수분 생기 채워요 감성 쿠굥 ...
4,뷰티컬리 페스타 역대급 세일 내일까지 모두 달려욧,미셸,2023. 2. 15. 21:24,etc.\n뷰티컬리 페스타 역대급 세일 내일까지 모두 달려욧\n미셸 ・ 2023. ...,"[뷰티컬리, 뷰티컬리페스타, 뷰티컬리세일, 뷰티컬리추천, 뷰티컬리추천템]",12.0,0.0,etc 뷰티 컬리 페스 타 역대 급 세 일 내일 까지 모두 달려 욧 미셸 2023 ...


In [5]:
beauty_df.to_csv('preprocessed_content_beauty.csv', encoding='utf-8', index=False)